In [ ]:
!pip install ankipandas

In [2]:
from ankipandas import Collection

In [3]:
col = Collection("/Users/abhi/Library/Application Support/Anki2")

INFO: Database found at '/Users/abhi/Library/Application Support/Anki2/Original/collection.anki2'.


In [4]:
nonsuspended = col.cards[col.cards['cqueue'] != 'suspended']

In [5]:
notes = nonsuspended.merge_notes().reset_index()

In [7]:
notes.head()

,cid,nid,cord,cmod,cusn,ctype,cqueue,cdue,civl,cfactor,...,cleft,codue,cdeck,codeck,nguid,nmod,nusn,ntags,nflds,nmodel
0,1402456305835,1402456236692,0,1633066125,355,relearn,due,860,45,2800,...,2002,0,AnKing Lolnotacop Bugs,,"b}T|#!(u,@",1629303869,206,[#AK_Step1_v10::#B&B::13_Infectious_Disease::0...,[{{c1::<u>Preformed</u> <b>enterotoxins</b>::W...,Cloze-AnKingMaster-v3
1,1501451382033,1402456236692,1,1633531091,399,relearn,due,869,48,2650,...,2002,0,AnKing Lolnotacop Bugs,,"b}T|#!(u,@",1629303869,206,[#AK_Step1_v10::#B&B::13_Infectious_Disease::0...,[{{c1::<u>Preformed</u> <b>enterotoxins</b>::W...,Cloze-AnKingMaster-v3
2,1501451638884,1402456236692,2,1633531191,399,relearn,due,866,45,2650,...,2002,0,AnKing Lolnotacop Bugs,,"b}T|#!(u,@",1629303869,206,[#AK_Step1_v10::#B&B::13_Infectious_Disease::0...,[{{c1::<u>Preformed</u> <b>enterotoxins</b>::W...,Cloze-AnKingMaster-v3
3,1402524178958,1402524036532,0,1632595032,315,relearn,due,828,18,2650,...,2002,0,AnKing Lolnotacop Bugs,,qs0X5x#LxS,1629303869,206,[#AK_Step1_v10::#B&B::13_Infectious_Disease::0...,[Where in the body are Group D Streptococci (<...,Cloze-AnKingMaster-v3
4,1402524202094,1402524179050,0,1633460676,392,relearn,due,850,30,2650,...,2002,0,AnKing Lolnotacop Bugs,,EEo(71fAQZ,1629303892,206,"[#AK_Step1_v10::#AMBOSS, #AK_Step1_v10::#B&B::...",[{{c1::<b>Subacute endocarditis</b>}} is a car...,Cloze-AnKingMaster-v3


In [8]:
revs = col.revs.reset_index()

In [9]:
notes.merge(revs, how='left', left_on="cid", right_on="cid").to_csv('out.csv')

In [10]:
# pick up from last point
import pandas as pd
import re

In [11]:
notes = pd.read_csv('out.csv')

In [26]:
def process_tags(tags):
    tags = tags.strip('][').split(',')
    new_tags = []
    for t in tags:
        try: 
            l = t.split('::')[-1]
            l = ''.join([i for i in l if not i.isnumeric()])
            if '#' not in l and len(l) > 5:
                
                new_tags.append(l.lower())
                
        except:
            pass
    return list(set(new_tags))


def cleanhtml(raw_html):
    CLEANR = re.compile('<.*?>') 
    cleantext = re.sub(CLEANR, '', raw_html)
    return cleantext

def process_fields(row):
    row = row.strip('][').split(',')
    tags = []
    try:
        pattern = re.compile(r'\{\{.*?\}\}')
        match = pattern.findall(row[0])
        
        for m in match:
            prelimtext =  m.split('::')[1]
            if 'img' not in prelimtext:
                s = cleanhtml(prelimtext).replace('}}', '')
                if len(s) > 4:
                    tags.append(s.lower())
            
            
        
    except:
        pass
    return list(set(tags))
def make_tags(row):
    try:
        tags = [] # process_tags(row['ntags'])
        fields = process_fields(row['nflds'])
        other_tags = []
        if 'DGSOM' in row['cdeck']:
            if 'faces' not in row['cdeck']:
                other = row['cdeck'].replace('\x1f', ' ').split(' ')
                for o in other:
                    if len(o) > 3 and ('DGSOM' not in o):
                        other_tags.append(o.lower())
        return list(set(tags + fields + other_tags))
    except:
        return []

In [27]:
notes.iloc[43203]

Unnamed: 0                                                43203
cid                                               1639442329162
nid                                               1639442329162
cord                                                          0
cmod                                                 1639588014
cusn                                                        564
ctype                                                   relearn
cqueue                                                      due
cdue                                                        892
civl                                                          1
cfactor                                                    2500
creps                                                         3
clapses                                                       0
cleft                                                      1001
codue                                                         0
cdeck                          DGSOMHis

In [28]:
notes['tags'] = notes.apply(make_tags, axis=1)

In [29]:
notes['creps'].value_counts()

4     8839
3     7743
5     5079
6     4248
7     3349
8     2945
9     2149
2     1615
10    1443
11    1167
1      793
12     780
13     760
14     503
15     354
16     294
0      275
17     242
18     147
20     123
21     108
23      95
19      76
28      28
27      27
25      25
Name: creps, dtype: int64

In [30]:
import networkx as nx

In [31]:
G = nx.Graph()
s = set()
for index, row in notes.iterrows():
    if index % 2000 == 0:
        print(index, G.number_of_edges())
    new_node = row['nid']
    new_attributes = row['tags']
    if new_node not in s:
        s.add(new_node)
        G.add_node(new_node, my_attributes=new_attributes)
        for node, attrs in G.nodes(data=True):
            if node != new_node:
                added = False
                for attr in attrs['my_attributes']:
                    if attr in new_attributes:
                        if added == False:
                            G.add_edge(node, new_node)
                            added = True
                        else:
                            break
                    

0 0
2000 251
4000 530
6000 737
8000 1022
10000 1277
12000 26281
14000 179824
16000 527223
18000 892286
20000 1331528
22000 2078698
24000 2758620
26000 3454260
28000 3810680
30000 4474775
32000 4954138
34000 5508984
36000 6198700
38000 6623312
40000 7338616
42000 8740465


In [32]:
G.number_of_nodes()

8167

In [33]:
G.number_of_edges()

10229426

In [34]:
sorted([len(x) for x in nx.connected_components(G)], reverse=True)[:10]

[6716, 8, 8, 8, 7, 7, 6, 5, 5, 5]

In [35]:
tree = nx.algorithms.tree.mst.minimum_spanning_tree(G)

In [36]:
G = tree

In [37]:
for component in list(nx.connected_components(G)):
    if len(component)<5:
        for node in component:
            G.remove_node(node)

In [38]:
G.number_of_nodes()

6785

In [39]:
G.number_of_edges()

6773

In [40]:
from networkx.readwrite import json_graph
import json

In [41]:
j = json_graph.node_link_data(G)

In [44]:
import copy

In [45]:
jnew = copy.deepcopy(j)

In [46]:
for idx,n in enumerate(jnew['nodes']):
    note_orig = notes[notes.nid == n['id']]
    time = note_orig.iloc[0]['rid']
    note = note_orig.iloc[0]['nflds'].strip('][').split(',')[0].replace('\'', '')
    jnew['nodes'][idx].pop('my_attributes', None)
    jnew['nodes'][idx]['note'] = note
    jnew['nodes'][idx]['time'] = time

In [47]:
import simplejson

In [48]:
with open('out_clean.json', 'w') as outfile:
    d = simplejson.dumps(jnew, ignore_nan=True)
    outfile.write(d)